## Imports

In [1]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration

### Scene imports

In [2]:
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import MeshcatVisualizerParams, Role, GeometrySet, CollisionFilterDeclaration
from pydrake.geometry.optimization import CspaceFreePolytope, SeparatingPlaneOrder
from pydrake.multibody.rational import RationalForwardKinematics

### Widgets imports

In [3]:
from ipywidgets import widgets
from functools import partial

## Set up the scene

In [4]:
meshcat = StartMeshcat()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

# Add the robot
gripper = parser.AddModels(file_name="my_sdfs/wsg_2dof.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]

# Set welds
plant.WeldFrames(
    plant.world_frame(), 
    plant.GetFrameByName("base_link", cap),
    RigidTransform(RotationMatrix(), [0, 0, 0]))

p_GgraspO = [0, 0, .07]
R_GgraspO = RotationMatrix.MakeXRotation(-np.pi / 2)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_wsg", gripper),
    RigidTransform(R_GgraspO, p_GgraspO))

plant.Finalize()

# Release collision constraints
inspector = scene_graph.model_inspector()

# Penetration allowed between gripper and cap

# Gripper inner collision
gripper_geometries = GeometrySet()
for i in range(len(plant.GetBodyIndices(gripper))):
    if i == 0: # Skip the base link
        continue
    body_index = plant.GetBodyIndices(gripper)[i]
    body_geometries = inspector.GetGeometries(
        plant.GetBodyFrameIdOrThrow(body_index))
    gripper_geometries.Add(geometry_ids=body_geometries)
scene_graph.collision_filter_manager().Apply(
    CollisionFilterDeclaration().ExcludeWithin(gripper_geometries))

# Gripper and cap collision
cap_geometries = GeometrySet()
for i in range(len(plant.GetBodyIndices(cap))):
    body_index = plant.GetBodyIndices(cap)[i]
    body_geometries = inspector.GetGeometries(
        plant.GetBodyFrameIdOrThrow(body_index))
    cap_geometries.Add(geometry_ids=body_geometries)

scene_graph.collision_filter_manager().Apply(
    CollisionFilterDeclaration().ExcludeBetween(gripper_geometries, cap_geometries))


# Add visualization
meshcat_params = MeshcatVisualizerParams()
meshcat_params.role = Role.kIllustration

visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    meshcat_params)

meshcat.SetProperty("/Background", "top_color", [0.8, 0.8, 0.6])
meshcat.SetProperty("/Background", "bottom_color", [0.9, 0.9, 0.9])

print(meshcat.web_url())
diagram = builder.Build()

# Context
diagram_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(diagram_context)
scene_graph_context = scene_graph.GetMyMutableContextFromRoot(diagram_context)


# Set initial configuration
plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.033
)
plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
    plant_context, 0.033
)


diagram = diagram.ForcedPublish(diagram_context)

# TCspace
Rat_FK = RationalForwardKinematics(plant)

print(plant.num_positions())

q_star = np.zeros(3)

cspace_free_polytope = CspaceFreePolytope(
    plant, 
    scene_graph,
    SeparatingPlaneOrder.kAffine,
    q_star)

INFO:drake:Meshcat listening for connections at http://localhost:7000


http://localhost:7000
4


## Set up the sliders so we can move the plant around manually

You can use the sliders below to move the three degrees of freedom of the plant around. A green dot will appear in the TC-space visualization describing the current TC-space configuration.

In [ ]:
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]
    q_high = plant.GetPositionUpperLimits()[i]
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, description=f"q{i}"))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

In [5]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

gripper = parser.AddModels(file_name="my_sdfs/wsg.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]

plant.Finalize()

In [6]:
B_cap = plant.GetBodyByName("cap_link", cap)
B_gripper = plant.GetBodyByName("body", gripper)


In [7]:
meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

In [ ]:
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

In [8]:
world_body = plant.world_body()
X_WO = plant.EvalBodyPoseInWorld(plant_context, world_body)

p_GgraspO = [0, .115, 0]
R_GgraspO = RotationMatrix.MakeXRotation(np.pi / 2).multiply(RotationMatrix.MakeZRotation(np.pi / 2))

X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
X_OGgrasp = X_GgraspO.inverse()
X_WGgrasp = X_WO.multiply(X_OGgrasp)

plant.SetFreeBodyPose(plant_context, B_gripper, X_WGgrasp)

plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.033
)
plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
    plant_context, 0.033
)

RuntimeError: A function call on a drake::multibody::MultibodyPlant<double> system named '::plant' was passed the Context of a system named '::_::plant' instead of the appropriate subsystem Context.
For more information about Context-System mismatches, see https://drake.mit.edu/troubleshooting.html#framework-context-system-mismatch

In [ ]:
diagram.ForcedPublish(context)